<a href="https://colab.research.google.com/github/abhilashhn1993/depression-assessment-on-twitter/blob/Dragon/Code/TweetCleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install spacy

In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import spacy
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

**BASIC TEXT CLEANING**
    

1.   Punctuation removal
2.   Removing URLs
3.   Removing Junk characters (non-english)
4.   Remove @ mentions from tweets
5.   Change to lower case   

In [0]:
#Method to remove texts with consecutive characters
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

In [0]:
def cleanTweets(df):
   # remove twitter Return handles (RT @xxx:)
   df['Tweets'] = np.vectorize(remove_pattern)(df['Tweets'], "RT @[\w]*:")
   # remove twitter handles (@xxx)
   df['Tweets'] = np.vectorize(remove_pattern)(df['Tweets'], "@[\w]*")
   # remove URL links (httpxxx)
   df['Tweets'] = np.vectorize(remove_pattern)(df['Tweets'], "https?://[A-Za-z0-9./]*")
   return df

In [6]:
from google.colab import files
uploaded = files.upload()

Saving Post_Tweets_All_users.xlsx to Post_Tweets_All_users.xlsx


In [0]:
import io
df_pre = pd.read_excel(io.BytesIO(uploaded['Dep_users_pre-post_all_tweets_with_senti_scores.xlsx']), sheet_name='pre_tweets_all_types')
df_post = pd.read_excel(io.BytesIO(uploaded['Post_Tweets_All_users.xlsx']))

In [0]:
df_pre.head()

In [0]:
df_pre['Tweets'] = df_pre['Tweets'].map(lambda x: str(x))
df_post['Tweets'] = df_post['Tweets'].map(lambda x: str(x))

In [0]:
df_pre_conc = df_pre.groupby(['Twitter_handle','User_type'])['Tweets'].agg(' '.join).reset_index(name='Tweets')
df_pre_conc.head()

In [9]:
df_post_conc = df_post.groupby(['Twitter_handle','User_type'])['Tweets'].agg(' '.join).reset_index(name='Tweets')
df_post_conc.head()

,Twitter_handle,Tweets
0,44Hayley_,Sheâ€™s so vocal Iâ€™m in the middle of a med...
1,6llvck,that was the worst dream iâ€™ve ever had just ...
2,8bitgrace,can you win me Steve from Minecraft *the most ...
3,A_Snippy_Snail,It creeps me out how you can pick up a story t...
4,AbbyZinger,omg yeah I wanted ALLL of the bingo squares I'...


In [0]:
df_pre_conc = cleanTweets(df_pre_conc)
df_post_conc = cleanTweets(df_post_conc)

In [0]:
regex = re.compile('[^a-zA-Z]')

In [0]:
#Remove non-alphanumeric values from pre data
for index,row in df_pre_conc.iterrows():
  out = regex.sub(' ',row['Tweets'])
  df_pre_conc.at[index, 'Tweets'] = ' '.join(out.split())

In [0]:
#Remove non-alphanumeric values from post data
for index,row in df_post_conc.iterrows():
  out = regex.sub(' ',row['Tweets'])
  df_post_conc.at[index, 'Tweets'] = ' '.join(out.split())

In [18]:
df_post_conc.head()

,Twitter_handle,Tweets
0,44Hayley_,She s so vocal I m in the middle of a med chan...
1,6llvck,that was the worst dream i ve ever had just pl...
2,8bitgrace,can you win me Steve from Minecraft the most p...
3,A_Snippy_Snail,It creeps me out how you can pick up a story t...
4,AbbyZinger,omg yeah I wanted ALLL of the bingo squares I ...


In [0]:
df_pre_conc.to_excel('pre_all_tweets_concat_cleaned.xlsx')
df_post_conc.to_excel('post_all_tweets_concat_cleaned.xlsx')

In [0]:
from google.colab import files
files.download('pre_all_tweets_concat_cleaned.xlsx')
files.download('post_all_tweets_concat_cleaned.xlsx')